In [1]:
import scvi

In [1]:
import sys
IN_COLAB = "google.colab" in sys.modules
def allow_notebook_for_test():
    print("Testing the totalVI notebook")
show_plot = True
test_mode = False
save_path = "data/"
if not test_mode:
    save_path = "../../data"
if IN_COLAB:
    !pip install --quiet git+https://github.com/yoseflab/scvi@dataset_refactor#egg=scvi[notebooks]
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import anndata
import os
from scvi.dataset import setup_anndata, highly_variable_genes_seurat_v3, pbmcs_10x_cite_seq
from scvi.models import TOTALVI
from scvi.inference import TotalPosterior, TotalTrainer
from scvi import set_seed
if IN_COLAB:
    %matplotlib inline
# Sets the random seed for torch and numpy
set_seed(0)
sc.set_figure_params(figsize=(4, 4))
adata = pbmcs_10x_cite_seq(save_path=save_path, run_setup_anndata= False)
highly_variable_genes_seurat_v3(adata, batch_key="batch", n_top_genes=4000, subset=True)
print(adata)
setup_anndata(adata, batch_key="batch", protein_expression_obsm_key="protein_expression")
sum_stats = adata.uns["scvi_summary_stats"]
use_cuda = True
lr = 4e-3

import time


/data1/galen/.pyenv/versions/miniconda3-latest/envs/scvi/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


[2020-08-06 22:14:07,109] INFO - scvi.dataset._preprocessing | added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)


/data1/galen/scVI/scvi/dataset/_preprocessing.py:166: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df.loc[: int(n_top_genes), "highly_variable"] = True


AnnData object with n_obs × n_vars = 10849 × 4000
    obs: 'n_genes', 'percent_mito', 'n_counts', 'batch', 'labels'
    var: 'encode', 'n_cells-0', 'highly_variable-0', 'hvg_encode-0', 'n_cells-1', 'highly_variable-1', 'hvg_encode-1', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'hvg'
    obsm: 'protein_expression'
[2020-08-06 22:14:07,585] INFO - scvi.dataset._anndata | Using batches from adata.obs["batch"]
[2020-08-06 22:14:07,588] INFO - scvi.dataset._anndata | No label_key inputted, assuming all cells have same label
[2020-08-06 22:14:07,590] INFO - scvi.dataset._anndata | Using data from adata.X
[2020-08-06 22:14:08,241] INFO - scvi.dataset._anndata | Computing library size prior per batch
[2020-08-06 22:14:08,360] INFO - scvi.dataset._anndata | Using protein expression from adata.obsm['protein_expression']
[2020-08-06 22:14:08,365] INFO - scvi.dataset._anndata | Using protein names from columns of adata.obs

/data1/galen/.pyenv/versions/miniconda3-latest/envs/scvi/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/data1/galen/.pyenv/versions/miniconda3-latest/envs/scvi/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [ ]:
times = []
n_epochs = 400
for i in range(5):
    totalvae = TOTALVI(
        sum_stats["n_genes"],
        sum_stats["n_proteins"],
        n_batch=sum_stats["n_batch"],
    )
    trainer = TotalTrainer(
        totalvae,
        adata,
        train_size=0.90,
        test_size=0.10,
        use_cuda=use_cuda,
        frequency=1,
        batch_size=256,
        early_stopping_kwargs="auto" if not test_mode else None
    )
    start = time.time()
    trainer.train(n_epochs=n_epochs)
    end = time.time()
    print(end-start)
    times.append(end-start)
# np.save('refactor_totalvi_{}_epochs.npy'.format(n_epochs), times)

[2020-08-06 23:13:45,832] INFO - scvi.inference.inference | KL warmup for 8136.75 iterations
training:  30%|███       | 121/400 [03:52<08:56,  1.92s/it]

In [5]:
np.save('refactor_totalvi_{}_epochs_2.npy'.format(n_epochs), times)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scvi
import scanpy as sc
if IN_COLAB:
    %matplotlib inline
# Sets torch and numpy random seeds, run after all scvi imports
scvi.set_seed(0)
gene_dataset = scvi.dataset.cortex(save_path=save_path)
# We do a standard normalization for scanpy downstream visualization
# Putting the count data in a layer preserves them for scVI
sc.pp.filter_genes(gene_dataset, min_cells=15)
gene_dataset.layers["counts"] = gene_dataset.X.copy()
sc.pp.normalize_total(gene_dataset, target_sum=1e4)
sc.pp.log1p(gene_dataset)
scvi.dataset.highly_variable_genes_seurat_v3(gene_dataset, n_top_genes=1000, layer="counts", subset=True)
scvi.dataset.setup_anndata(gene_dataset, X_layers_key="counts")
n_epochs = 400
lr = 1e-3
use_cuda = True
ss = gene_dataset.uns["scvi_summary_stats"]
vae = scvi.models.VAE(ss["n_genes"])
trainer = scvi.inference.UnsupervisedTrainer(
    vae,
    gene_dataset,
    train_size=0.90,
    use_cuda=use_cuda,
    frequency=5,
)
import time

for n_epochs in [50,100,200]
    times = []
    for i in range(55):
        start = time.time()
        trainer.train(n_epochs=n_epochs)
        end = time.time()
        print(end-start)
        times.append(end-start)

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scvi
import scanpy as sc

# Sets torch and numpy random seeds, run after all scvi imports
scvi.set_seed(0)
gene_dataset = scvi.dataset.cortex(save_path='data')
# We do a standard normalization for scanpy downstream visualization
# Putting the count data in a layer preserves them for scVI
sc.pp.filter_genes(gene_dataset, min_cells=15)

# sc.pp.normalize_total(gene_dataset, target_sum=1e4)
# sc.pp.log1p(gene_dataset)
scvi.dataset.highly_variable_genes_seurat_v3(gene_dataset, n_top_genes=1000, subset=True)


[2020-08-11 20:07:50,853] INFO - scvi.dataset._built_in_data._utils | File /data1/galen/scVI/galen/data/expression.bin already downloaded
[2020-08-11 20:07:50,855] INFO - scvi.dataset._built_in_data._cortex | Loading Cortex data from /data1/galen/scVI/galen/data/expression.bin
[2020-08-11 20:08:07,611] INFO - scvi.dataset._built_in_data._cortex | Finished loading Cortex data
[2020-08-11 20:08:07,998] INFO - scvi.dataset._anndata | Using batches from adata.obs["batch"]


/data1/galen/.pyenv/versions/miniconda3-latest/envs/scvi/lib/python3.8/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


[2020-08-11 20:08:08,000] INFO - scvi.dataset._anndata | Using labels from adata.obs["labels"]
[2020-08-11 20:08:08,002] INFO - scvi.dataset._anndata | Using data from adata.X
[2020-08-11 20:08:08,989] INFO - scvi.dataset._anndata | Computing library size prior per batch


/data1/galen/scVI/scvi/dataset/_anndata.py:438: UserWarning: adata has more genes than cells. SCVI may not work properly.
  warnings.warn("adata has more genes than cells. SCVI may not work properly.")
/data1/galen/.pyenv/versions/miniconda3-latest/envs/scvi/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


[2020-08-11 20:08:09,327] INFO - scvi.dataset._anndata | Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']
[2020-08-11 20:08:09,329] INFO - scvi.dataset._anndata | Successfully registered anndata object containing 3005 cells, 19972 genes, 1 batches, and 0 proteins.
[2020-08-11 20:08:11,804] INFO - scvi.dataset._preprocessing | added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)


/data1/galen/scVI/scvi/dataset/_preprocessing.py:166: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df.loc[: int(n_top_genes), "highly_variable"] = True


In [4]:
from scipy.sparse import csr_matrix

In [5]:
gene_dataset.X = csr_matrix(gene_dataset.X)

In [4]:
#gene_dataset.layers['raw']

<3005x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 469008 stored elements in Compressed Sparse Row format>

In [6]:
scvi.dataset.setup_anndata(gene_dataset)


[2020-08-11 20:08:12,123] INFO - scvi.dataset._anndata | No batch_key inputted, assuming all cells are same batch
[2020-08-11 20:08:12,125] INFO - scvi.dataset._anndata | No label_key inputted, assuming all cells have same label
[2020-08-11 20:08:12,127] INFO - scvi.dataset._anndata | Using data from adata.X
[2020-08-11 20:08:12,145] INFO - scvi.dataset._anndata | Computing library size prior per batch
[2020-08-11 20:08:12,159] INFO - scvi.dataset._anndata | Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']
[2020-08-11 20:08:12,160] INFO - scvi.dataset._anndata | Successfully registered anndata object containing 3005 cells, 1000 genes, 1 batches, and 0 proteins.


/data1/galen/.pyenv/versions/miniconda3-latest/envs/scvi/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [7]:
gene_dataset.uns['scvi_data_registry']

{'X': ['_X', None],
 'batch_indices': ['obs', '_scvi_batch'],
 'local_l_mean': ['obs', '_scvi_local_l_mean'],
 'local_l_var': ['obs', '_scvi_local_l_var'],
 'labels': ['obs', '_scvi_labels']}

In [ ]:
lr = 1e-3
use_cuda = False
ss = gene_dataset.uns["scvi_summary_stats"]

import time

for n_epochs in [50,100,200,400]:
    times = []
    for i in range(5):
        vae = scvi.models.VAE(ss["n_genes"])
        trainer = scvi.inference.UnsupervisedTrainer(
            vae,
            gene_dataset,
            train_size=0.90,
            use_cuda=use_cuda,
            frequency=5,
        )
        start = time.time()
        trainer.train(n_epochs=n_epochs)
        end = time.time()
        print(end-start)
        times.append(end-start)
    np.save('A_refactor_scvi_{}_epochs_cpu_X_sparse.npy'.format(n_epochs), times)

[2020-08-11 20:08:37,209] INFO - scvi.inference.inference | KL warmup phase exceeds overall training phaseIf your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.
[2020-08-11 20:08:37,210] INFO - scvi.inference.inference | KL warmup for 400 epochs
training: 100%|██████████| 50/50 [00:30<00:00,  1.64it/s]
[2020-08-11 20:09:07,688] INFO - scvi.inference.inference | Training is still in warming up phase. If your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.
30.749072790145874
[2020-08-11 20:09:08,013] INFO - scvi.inference.inference | KL warmup phase exceeds overall training phaseIf your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.
[2020-08-11 20:09:08,014] INFO - scvi.inference.inference | KL warmup for 400 epochs
training: 100%|██████████| 50/50 [00:30<00:00,  1.66it/s]
[2020-08-11 20:09:38,126] INFO - scvi.inferen

In [6]:
np.save('refactor_scvi_{}_epochs.npy'.format(n_epochs), times)

In [9]:
gene_dataset.X.shape

(3005, 1000)